# Python NLP Tutorial 

## Introduction

This tutorial was designed as a training tool for the HERC NLP Team and serves as a basic introduction to:

* Python programming concepts
* Implementation of natural language processing techniques (NLP)
* Popular Python libraries and data constructs used in data analytics
* NLP applications towards healthcare

This tutorial assumes basic understanding of programming techniques, syntax, and structures, whether in Python or another language.

### Tools

This tutorial uses an application called [Jupyter Notebook](https://jupyter.org) as the programming interface. Jupyter is an interactive development environment that allows users to create and modify dynamic code "cells" and run them individually. This allows for easy experimentation and testing of code. The cells below have instructions and guidelines on how to manipulate the code for various results. You are also free to experiment on your own.

### Notes

* Each of these sections of text or code represent a cell. You will see the cell outlined when you click on it.
* Any text that is [underlined]() is a link, usually to a reference page that provides more information about a given module or function.
* A cell can be run by pressing `Ctrl + Enter` or by clicking the "Run" button on the toolbar above.

## Step 1: Imports

The first step in a Python program is to declare any external modules that will be used by the code. A module is a name for any file that contains Python code. A group of modules is often referred to as a library. A lot of the work we need has already been accomplished by others, so we can just download and reference their code (modules) for our own programs. We will be using the following modules:

### re
The [`re`](https://docs.python.org/3/library/re.html) module will allow us to search text using regular expressions, an approach to pattern recognition. These will be explained in greater depth below.

### pandas
The [`pandas`](https://pandas.pydata.org) module provides objects and methods that will allow us to efficiently structure and analyze large data sets. The primary structure is called a DataFrame, a 2D tabular object that allows for high performance of complex operations. A user-friendly tutorial can be found here: https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python

In [ ]:
# Declare an import using the keyword 'import'
# Click on this cell to select it and press 'ctrl + enter' to run

import re
import pandas as pd  # we use the 'as' keyword to declare a shortcut to reference the pandas module

## Step 2: Data Input

The first step to analyzing data is to input the data. We will be working with an extract from the [FDA MAUDE database](https://www.fda.gov/medical-devices/mandatory-reporting-requirements-manufacturers-importers-and-device-user-facilities/manufacturer-and-user-facility-device-experience-database-maude), which features adverse event reports involving medical devices. We are interested in reports related to EPIC software, so we have pulled a dataset of reports that contain the word "EPIC".

We will read our data from a CSV file ('EPIC_MAUDE.csv') and load it into our program using the [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method. This will translate the file into a DataFrame that can be accessed for our analysis.

After we read the data, we can view the output to make sure it's formatted correctly. In Jupyter Notebook, the final command of a cell will display its output (if applicable) every time you run that cell. We thus use the final command in the below cell to display the data using bracket notation. Python, like most mainstream languages, is 0-indexed, meaning that the first element of a list is referenced with the number \[0\], the second with \[1\], and so on. With a pandas DataFrame, we can access a subsection, or "slice" of the data, using colon notation `[start_index:stop_index]`. Play around with the example below using different values and observe what the output looks like.

In [ ]:
# if you get an error running this cell, make sure you've run the imports cell first

data = pd.read_csv('EPIC_MAUDE.csv')
#print(data[0:10])  # uncomment this line by removing the first '#' symbol at the beginning
data[0:10]  # try entering different numbers and re-running.

Alternatively, we can use the [`print`](https://docs.python.org/3/library/functions.html#print) function to output data from any part of a cell. Try to uncomment the line `print(data[0:10])` in the cell above and re-run. 

## Step 3: Explore the Data

### Regular Expressions Intro
The next step is to closer examine the data. We do some preliminary investigations to see what is notable or interesting about the data, anything that stands out or presents a feasible direction for our analysis. For quantitative data, this would often involve descriptive statistics, histograms, and the like, but for text based data, the approach is more driven by the content of the data or the type of problem we are trying to solve. In this case, we are looking at reports related to EPIC, so let's look for occurrences of the word "Epic".

We will use a search pattern technique called [regular expressions](https://en.wikipedia.org/wiki/Regular_expression). The [`search`](https://docs.python.org/3/library/re.html#re.search) function allows us to search a text string for a given pattern. It has two required arguments: 1) the search pattern and 2) the text to be searched. It then returns a [match object](https://docs.python.org/3/library/re.html#match-objects) that contains information about any matches in the text string. Searching is case sensitive by default, so we can circumvent this issue by making our pattern and search text both lowercase. In the cell below, we set our pattern to search for the string "epic". We will select the first record in our data file as the search string. Run the cells below.

In [ ]:
pattern = 'epic'
text = data['ed_text'][0].lower()  # notation to read the first record of the 'ed_text' column and lowercase the text
match = re.search(pattern, text)
print(match)

### Using Match Objects

When the code above runs, it prints the match object that is returned. The segment `span=(316, 320)` tells us that the search pattern was located between character 316 and 320 in text.

For this to be practically useful, we can create a function that prints the surrounding text, just to give us the context that the word was used in. It will take two inputs, 1) a match object and 2) the window size (denoting how much surrounding text to show). The function will then output a string that contains the matched word and its context based on the inputs. We can then use a loop to examine several records this way.

In [ ]:
def context(match, window=25):  # window size variable defaults to 25 if not explicitly given
    start_index = max(0, match.start() - window)  # to prevent a negative number assignment
    end_index = match.end() + window
    
    return match.string[start_index:end_index]

print(context(match, 20))  # try different values for the window size variable to see what a helpful window looks like

In [ ]:
pattern = 'epic'
matches = []  # create an empty list

# iterate through reports and return a list of match objects (for positive matches) 
for text in data['ed_text']:  # loop through entire 'ed_text' column
    match = re.search(pattern, text.lower())
    if match is not None:
        matches.append(match)  # add the match object to the list
            
for match in matches[:25]:  # loops through first 25 matches, change this value if you like
    print(context(match))  # add a second argument for window size or change the default (and rerun the cell) in the function above

### Going Deeper, Part I

We can immediately see that there are several reports we are not interested in. Since we searched for the term "epic", we are seeing reports related to "Epic vascular stent system" and other types of medical devices. Here is where we begin to see the power of regular expressions. We can use them to make our search more precise, so it picks up occurrences of the word "epic" that are not followed by terms related to the vascular stent system. Looking at the output from above, we can see a few words we may want to filter out-- "vascular", "stent", "valve", etc.

We can get this done using a _negative lookahead assertion_, which matches if the following text does not match the given expression. This means we can look for text that matches "epic", but does _not_ match "vascular", "stent", or "valve". The syntax for this assertion is `(?!<expression>)`. We can identify multiple words to avoid using the `|` character. **Replace line 1 in the above code cell** with

    pattern = 'epic (?!stent|valve|vascular|sds|implant)'
    
and rerun the code to see what happens. Try to add or remove terms from the expression to see how it changes the results.

Additional resources:
* a [reference page](https://docs.python.org/3/library/re.html#regular-expression-syntax) for regular expression syntax.
* a [helpful tutorial](https://www.datacamp.com/community/tutorials/python-regular-expression-tutorial) on regular expressions in Python.

### Going Deeper, Part II

You can see that the negative lookahead assertion removed some of the unwanted reports, but still leaves a lot of noise. We can keep adding terms to the assertion but if we look closely at the results, we see that there are often additional words between "epic" and one of our filter terms (e.g. "epic self-expanding stents", "epic heart valve"). So a cleaner approach would be to account for this word gap, regardless of what the interceding words are. A quick approach is to add a secondary filter step, that would remove reports that contain one of our "negative" keywords nearby the word "epic".

In this case, we are creating a secondary pattern that, if matched, will pass over that report instead of adding it to our list of matches. The code for this algorithm is below. Note line 2:

    negative_pattern = 'epic ([^.!?]+ ){0,5}?(stent|valve|vascular|sds|implant)'

This is beginning to look pretty complicated, so let's break it down.
* We first have `epic `, which, as before, looks for the word "epic", followed by a space.
* Next is this bizarre pattern `([^.!?]* )`. This essentially matches a word, excluding any end punctuation.
* The following `{0,5}?` tells it to match the previous pattern, `([^.!?]* )` anywhere between 0 and 5 times. This allows between zero and five words between the word "epic" and one of our exclusionary words, as long as there is no end punctuation.
* Finally, we have our list of exclusionary words `(stent|valve|vascular|sds|implant)`.

You can find a helpful demonstration [here](https://www.regexpal.com/?fam=111616), which highlights matched terms from a selection of reports. The link also allows you to edit the expression and see how the matched results change.

In [ ]:
positive_pattern = 'epic'
negative_pattern = 'epic ([^.!? ]+ ){0,5}?(stent|valve|vascular|sds|implant)'
matches = []

# iterate through reports and return a list of match objects (for positive matches) 
for text in data['ed_text']:
    match = re.search(positive_pattern, text.lower())
    if match is not None:
        if not re.search(negative_pattern, text.lower()):
            matches.append(match)  # add the match object to the list
                        
for match in matches[:25]:
    print(context(match, 50))

## Step 4: Output the Data
As you can see in the output above, there are still some irrelevant results, but the noise has been greatly reduced, and we can see more results related to the Epic system. There is definitely more filtering that can be done, so feel free to experiment some more. The final step is to 1) create a set of reports that meet our filtering requirements and 2) output these reports as a file. The `pandas` library provides easy functionality for these tasks.

1. First, we create a function that takes a text string and returns True or False depending on whether or not it passes our test filter.
2. Second, we use the [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.apply.html) function to create a new dataset comprised of all rows where the `ed_text` field passes our function.
3. Finally, we write our filtered dataset to a CSV file using the [`to_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) function.

Run the following cell. If you look at the output file, you will see that we've reduced our set from 1500 reports to 196! From here, we can tweak our code further, create additional filters, or start producing a training set for a machine learning algorithm. This code can be applied to any additional dataset that is similar to the original input.

**Note**: If you a running this notebook on the server (using Binder), you can access the output file by going back to the directory (where you opened this file) and clicking on the file. Once the file is open, you can go to "File" > "Download" to save a copy to your computer. You can also download this notebook as a Python file by going to "File" > "Download as" > "Python"

In [ ]:
### Step 1 ###
def is_match(text):
    text = text.lower()
    positive_pattern = 'epic'
    negative_pattern = 'epic ([^.!? ]+ ){0,5}?(stent|valve|vascular|sds|implant)'
    
    if re.search(positive_pattern, text):  # check for match of term "epic"
        if not re.search(negative_pattern, text.lower()):  # check against exclusionary pattern
            return True  # indicates that the text passes the filter
        
    return False  # default to False if text does not pass filter

### Step 2 ###
filtered_data = data[data.ed_text.apply(lambda x: is_match(x))]
# add the following code in the line below to check the output: print(filtered_data)

### Step 3 ###
filtered_data.to_csv('EPIC_MAUDE_filtered.csv', index=False)  # feel free to change the output name/location